In [ ]:
# ------------------------------------------------------------
# Summer NDVI median (single year), Landsat 8 SR L2, 30 m
# Per-state exports
# ------------------------------------------------------------

import ee
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# 0) Local paths (explicit, no src.paths)
# ------------------------------------------------------------
REPO_ROOT = Path(r"C:\Users\bdevoe\Desktop\Greenspace")

STATE_FIPS_FILE = REPO_ROOT / "resources" / "StateFipsUsps.csv"

# NDVI will be exported to your Google Drive folder:
DRIVE_FOLDER = "NDVI_DOWNLOAD_2020"

# ------------------------------------------------------------
# 1) Parameters
# ------------------------------------------------------------
YEAR = 2020                 # <---- change this for different year
SUMMER_START = "06-01"
SUMMER_END   = "09-01"

DATASET_ID = "LANDSAT/LC08/C02/T1_L2"   # Landsat 8 SR L2
SCALE_M = 30

# Landsat SR scaling
SR_SCALE  = 0.0000275
SR_OFFSET = -0.2

# ------------------------------------------------------------
# 2) Earth Engine init
# ------------------------------------------------------------
ee.Initialize()

# ------------------------------------------------------------
# 3) Build STATE LIST from your local CSV
# ------------------------------------------------------------
state_map_df = pd.read_csv(STATE_FIPS_FILE, dtype=str)
STATES = state_map_df["USPS"].str.upper().tolist()

# Example: if you ONLY want Lower-48 + DC, uncomment:
EXCLUDE = {"AK", "HI", "PR", "VI", "GU", "AS", "MP"}
STATES = [s for s in STATES if s not in EXCLUDE]

# Example: if you ONLY want AK + HI, uncomment:
# STATES = ["AK", "HI"]
# STATES = ["MA", "RI", "CT", "NY", "NJ", "PA", "ME", "VT", "NH"]

print("Exporting states:", STATES)

# ------------------------------------------------------------
# 4) Get TIGER state geometries
# ------------------------------------------------------------
# Recommend TIGER/2020 for consistency with your tract shapefiles
states_fc = (
    ee.FeatureCollection("TIGER/2018/States")
    .filter(ee.Filter.inList("STUSPS", STATES))
)

# ------------------------------------------------------------
# 5) Build single-year summer NDVI image
# ------------------------------------------------------------
start = f"{YEAR}-{SUMMER_START}"
end   = f"{YEAR}-{SUMMER_END}"

ic = (
    ee.ImageCollection(DATASET_ID)
    .filterDate(start, end)
)

# Cloud + shadow mask
def mask_cloud_shadow(img):
    qa = img.select("QA_PIXEL")
    cloud  = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    mask = cloud.And(shadow)
    return img.updateMask(mask)

ic = ic.map(mask_cloud_shadow)

# Scale SR bands
def scale_sr(img):
    scaled = img.select("SR_B.*").multiply(SR_SCALE).add(SR_OFFSET)
    return img.addBands(scaled, overwrite=True)

ic = ic.map(scale_sr)

# Add NDVI band (float)
def add_ndvi(img):
    ndvi = img.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    return img.addBands(ndvi)

ic = ic.map(add_ndvi)

# Median summer NDVI (float)
ndvi_summer = ic.select("NDVI").median().rename("NDVI")

# ------------------------------------------------------------
# 6) Per-state exports
# ------------------------------------------------------------
for st in STATES:
    geom = states_fc.filter(ee.Filter.eq("STUSPS", st)).geometry()
    filename = f"{st}_{YEAR}_summer_ndvi_{SCALE_M}m"

    print(f"Queueing export for: {filename}.tiff")

    ee.batch.Export.image.toDrive(
        image           = ndvi_summer.clip(geom),
        description     = filename,
        folder          = DRIVE_FOLDER,
        fileNamePrefix  = filename,
        region          = geom,
        scale           = SCALE_M,
        maxPixels       = 1e13,
        fileFormat      = "GeoTIFF",
        formatOptions   = {"cloudOptimized": True},
        # OPTIONAL but recommended for consistent tile layout:
        # fileDimensions  = 8192,
    ).start()

print(f"All state exports submitted to Drive/{DRIVE_FOLDER}.")


Exporting states: ['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
Queueing export for: AL_2025_summer_ndvi_30m.tiff
Queueing export for: AZ_2025_summer_ndvi_30m.tiff
Queueing export for: AR_2025_summer_ndvi_30m.tiff
Queueing export for: CA_2025_summer_ndvi_30m.tiff
Queueing export for: CO_2025_summer_ndvi_30m.tiff
Queueing export for: CT_2025_summer_ndvi_30m.tiff
Queueing export for: DE_2025_summer_ndvi_30m.tiff
Queueing export for: DC_2025_summer_ndvi_30m.tiff
Queueing export for: FL_2025_summer_ndvi_30m.tiff
Queueing export for: GA_2025_summer_ndvi_30m.tiff
Queueing export for: ID_2025_summer_ndvi_30m.tiff
Queueing export for: IL_2025_summer_ndvi_30m.tiff
Queueing export for: IN_2025_summer_ndvi_30m.tiff
Queueing export for: IA_2025_summer_n